In [ ]:
# import os
os.environ["MKL_NUM_THREADS"] = "1" 
os.environ["NUMEXPR_NUM_THREADS"] = "1" 
os.environ["OMP_NUM_THREADS"] = "1"
from os import path, listdir
import numpy as np
np.random.seed(1)
import random
random.seed(1)
import pandas as pd
import timeit
import cv2
from tqdm import tqdm

import sys
sys.setrecursionlimit(10000)
from multiprocessing import Pool

from shapely.geometry.linestring import LineString
# from skimage.morphology import skeletonize_3d, square, erosion, dilation, medial_axis
# from skimage.measure import label, regionprops, approximate_polygon
from math import hypot, sin, cos, asin, acos, radians
from sklearn.neighbors import KDTree
from shapely.wkt import dumps, loads

import scipy

import utm
#pip install utm
from osgeo import gdal
gdal.UseExceptions()
from osgeo import osr
from osgeo import ogr
#conda install gdal

import ntpath
from shapely.geometry import mapping, Point, LineString

# import matplotlib.pyplot as plt
# import seaborn as sns

pred_folders = ['./wdata/test_pred']

speed_bins = np.array([15, 18.75, 20, 25, 30, 35, 45, 55, 65])

test_folders = ['/fs/scratch/PCON0003/osu10670/AOI_7_Moscow_test', '/fs/scratch/PCON0003/osu10670/AOI_8_Mumbai_test']
# test_folders = []
# for i in range(1, len(sys.argv) - 1):
#     test_folders.append(sys.argv[i])

df = pd.read_csv(path.join('./wdata', 'solution_length.csv'), header=None)
df.columns = ['ImageId', 'WKT_Pix']

# example GDAL error handler function
def gdal_error_handler(err_class, err_num, err_msg):
    errtype = {
            gdal.CE_None:'None',
            gdal.CE_Debug:'Debug',
            gdal.CE_Warning:'Warning',
            gdal.CE_Failure:'Failure',
            gdal.CE_Fatal:'Fatal'
    }
    err_msg = err_msg.replace('\n',' ')
    err_class = errtype.get(err_class, 'None')
    print('Error Number: ', (err_num))
    print('Error Type: ', (err_class))
    print('Error Message: ', (err_msg))
    
gdal.PushErrorHandler(gdal_error_handler)

# from https://github.com/CosmiQ/cresi
def get_linestring_midpoints(geom):
    '''Get midpoints of each line segment in the line.
    Also return the length of each segment, assuming cartesian coordinates'''
    coords = list(geom.coords)
    N = len(coords)
    x_mids, y_mids, dls = [], [], []
    for i in range(N-1):
        (x0, y0) = coords[i]
        (x1, y1) = coords[i+1]
        x_mids.append(np.rint(0.5 * (x0 + x1)))
        y_mids.append(np.rint(0.5 * (y0 + y1)))
        dl = scipy.spatial.distance.euclidean(coords[i], coords[i+1])
        dls. append(dl)
    return np.array(x_mids).astype(int), np.array(y_mids).astype(int), \
                np.array(dls)


def pixelToGeoCoord(xPix, yPix, inputRaster, sourceSR='', geomTransform='', targetSR=''):
    '''from spacenet geotools'''

    if targetSR =='':
        performReprojection=False
        targetSR = osr.SpatialReference()
        targetSR.ImportFromEPSG(4326)
    else:
        performReprojection=True

    if geomTransform=='':
        srcRaster = gdal.Open(inputRaster)
        geomTransform = srcRaster.GetGeoTransform()

        source_sr = osr.SpatialReference()
        source_sr.ImportFromWkt(srcRaster.GetProjectionRef())

    geom = ogr.Geometry(ogr.wkbPoint)
    xOrigin = geomTransform[0]
    yOrigin = geomTransform[3]
    pixelWidth = geomTransform[1]
    pixelHeight = geomTransform[5]

    xCoord = (xPix * pixelWidth) + xOrigin
    yCoord = (yPix * pixelHeight) + yOrigin
    geom.AddPoint(xCoord, yCoord)

    if performReprojection:
        if sourceSR=='':
            srcRaster = gdal.Open(inputRaster)
            sourceSR = osr.SpatialReference()
            sourceSR.ImportFromWkt(srcRaster.GetProjectionRef())
        coord_trans = osr.CoordinateTransformation(sourceSR, targetSR)
        geom.Transform(coord_trans)

    return (geom.GetX(), geom.GetY())


def convert_pix_lstring_to_geo(wkt_lstring, im_file, 
                               utm_zone=None, utm_letter=None, verbose=False):
    '''Convert linestring in pixel coords to geo coords
    If zone or letter changes inthe middle of line, it's all screwed up, so
    force zone and letter based on first point
    (latitude, longitude, force_zone_number=None, force_zone_letter=None)
    Or just force utm zone and letter explicitly
        '''
    shape = wkt_lstring  #shapely.wkt.loads(lstring)
    x_pixs, y_pixs = shape.coords.xy
    coords_latlon = []
    coords_utm = []
    for i,(x,y) in enumerate(zip(x_pixs, y_pixs)):
        
        targetSR = osr.SpatialReference()
        targetSR.ImportFromEPSG(4326)
        lon, lat = pixelToGeoCoord(x, y, im_file, targetSR=targetSR)

        if utm_zone and utm_letter:
            [utm_east, utm_north, _, _] = utm.from_latlon(lat, lon,
                force_zone_number=utm_zone, force_zone_letter=utm_letter)
        else:
            [utm_east, utm_north, utm_zone, utm_letter] = utm.from_latlon(lat, lon)
        
        if verbose:
            print("lat lon, utm_east, utm_north, utm_zone, utm_letter]",
                [lat, lon, utm_east, utm_north, utm_zone, utm_letter])
        coords_utm.append([utm_east, utm_north])
        coords_latlon.append([lon, lat])
    
    lstring_latlon = LineString([Point(z) for z in coords_latlon])
    lstring_utm = LineString([Point(z) for z in coords_utm])
    
    return lstring_latlon, lstring_utm, utm_zone, utm_letter

meters_to_miles = 0.000621371

###########


def get_linestring_keypoints(geom):
    coords = list(geom.coords)
    N = len(coords)
    xs, ys, dls = [], [], []
    for i in range(N-1):
        xs.append([])
        ys.append([])

        (x0, y0) = coords[i]
        (x1, y1) = coords[i+1]
        
        xs[i].append(0.5 * x0 + 0.5 * x1)
        ys[i].append(0.5 * y0 + 0.5 * y1)
        xs[i].append(0.75 * x0 + 0.25 * x1)
        ys[i].append(0.75 * y0 + 0.25 * y1)
        xs[i].append(0.25 * x0 + 0.75 * x1)
        ys[i].append(0.25 * y0 + 0.75 * y1)
        xs[i].append(0.9 * x0 + 0.1 * x1)
        ys[i].append(0.9 * y0 + 0.1 * y1)
        xs[i].append(0.1 * x0 + 0.9 * x1)
        ys[i].append(0.1 * y0 + 0.9 * y1)
        xs[i].append(0.35 * x0 + 0.65 * x1)
        ys[i].append(0.35 * y0 + 0.65 * y1)
        xs[i].append(0.65 * x0 + 0.35 * x1)
        ys[i].append(0.65 * y0 + 0.35 * y1)

        dl = scipy.spatial.distance.euclidean(coords[i], coords[i+1])
        dls. append(dl)
    return xs, ys, np.asarray(dls)



def process_file(fn):
    
    img_id = ntpath.basename(fn)[0:-4]
    img_id = img_id.replace('_PS-MS', '')
    print(img_id)
    
    im_file = fn
    
    msks = []
    for pred_folder in pred_folders:
        msk0 = cv2.imread(path.join(pred_folder, img_id + '_speed0.png'), cv2.IMREAD_UNCHANGED)
        msk1 = cv2.imread(path.join(pred_folder, img_id + '_speed1.png'), cv2.IMREAD_UNCHANGED)
        msk2 = cv2.imread(path.join(pred_folder, img_id + '_speed2.png'), cv2.IMREAD_UNCHANGED)
        msk = np.concatenate((msk0, msk1, msk2), axis=2)
        if msk.shape[0] < 1306:
            msk = cv2.resize(msk, (1300, 1300))
            msk = np.pad(msk, ((6, 6), (6, 6), (0, 0)), mode='reflect')
        msks.append(msk)
    msks = np.asarray(msks)
    msk = msks.mean(axis=0)
    msk = msk[6:1306, 6:1306].astype('uint8')
    
    vals = df[(df['ImageId'] == img_id)]['WKT_Pix'].values
    res_rows = []
    for v in vals:
        if v == 'LINESTRING EMPTY':
            return [{'ImageId': img_id, 'WKT_Pix': 'LINESTRING EMPTY', 'length_m': 0, 'travel_time_s': 0}]
        
        l = loads(v)
        
        lstring_latlon, lstring_utm, utm_zone, utm_letter = convert_pix_lstring_to_geo(l, im_file)
    
        length = lstring_utm.length
        length_miles = length * meters_to_miles
        
#         x_mids, y_mids, dls = get_linestring_midpoints(l)
        xs, ys, dls = get_linestring_keypoints(l)
        
        _sz = 4

#         speed = []
#         if x_mids.shape[0] > 0:
#             for i in range(x_mids.shape[0]):
#                 x0 = max(0, x_mids[i] - _sz)
#                 x1= min(1300, x_mids[i] + _sz)
#                 y0 = max(0, y_mids[i] - _sz)
#                 y1= min(1300, y_mids[i] + _sz)
#                 patch = msk[y0:y1, x0:x1]
#                 means = patch.mean(axis=(0, 1))
                
#                 if means.sum() == 0:
#                     speed.append(25)
#                 else:
#                     means /= means.sum()
#                     _s = (speed_bins * means).sum()
#                     if _s < 15:
#                         _s = 15
#                     if _s > 65:
#                         _s = 65
#                     speed.append(_s)
                    
        speed = []
        if len(xs) > 0:
            for i in range(len(xs)):
                seg_speeds = []
                for j in range(len(xs[i])):
                    x0 = max(0, int(xs[i][j] - _sz))
                    x1= min(1300, int(xs[i][j] + _sz))
                    y0 = max(0, int(ys[i][j] - _sz))
                    y1= min(1300, int(ys[i][j] + _sz))
                    
                    patch = msk[y0:y1, x0:x1]
                    means = patch.mean(axis=(0, 1))

                if means.sum() == 0:
                    seg_speeds.append(25)
                else:
                    means /= means.sum()
                    _s = (speed_bins * means).sum()
                    if _s < 15:
                        _s = 15
                    if _s > 65:
                        _s = 65
                    seg_speeds.append(_s)
                speed.append(np.mean(seg_speeds))
                    
        speed = np.asarray(speed)
        dls /= dls.sum()
        speed = (speed * dls).sum()
        
        if speed < 15:
            speed = 15
        if speed > 65:
            speed = 65
                        
        hours = length_miles / speed
        
        travel_time_s = np.round(3600. * hours, 3)
        
        res_rows.append({'ImageId': img_id, 'WKT_Pix': v, 'length_m': length, 'travel_time_s': travel_time_s})


    return res_rows





In [15]:
# if __name__ == '__main__':
t0 = timeit.default_timer()

#     out_file = sys.argv[-1]

out_file = 'solution.csv'

all_files = []
for d in test_folders:
    for f in listdir(path.join(d, 'PS-MS')):
        if '.tif' in f:
            all_files.append(path.join(d+'/PS-MS',f))

#     for fn in tqdm(all_files):
#         process_file(fn)



In [3]:
a = process_file(all_files[0])

SN5_roads_test_public_AOI_7_Moscow_chip101


[{'ImageId': 'SN5_roads_test_public_AOI_7_Moscow_chip101',
  'WKT_Pix': 'LINESTRING (32 130, 33 136, 42 176, 66 254, 70 281, 91 375, 143 547, 173 672, 196 757, 215 816, 225 863, 275 1006, 287 1062, 291 1095, 293 1109)',
  'length_m': 343.03892300344666,
  'travel_time_s': 27.517},
 {'ImageId': 'SN5_roads_test_public_AOI_7_Moscow_chip101',
  'WKT_Pix': 'LINESTRING (293 1109, 318 1212, 332 1259, 334 1265)',
  'length_m': 54.622218692518075,
  'travel_time_s': 5.405}]

In [ ]:
with Pool() as pool:
    results = pool.map(process_file, all_files)



SN5_roads_test_public_AOI_7_Moscow_chip101SN5_roads_test_public_AOI_7_Moscow_chip164SN5_roads_test_public_AOI_7_Moscow_chip171SN5_roads_test_public_AOI_7_Moscow_chip120SN5_roads_test_public_AOI_7_Moscow_chip65

SN5_roads_test_public_AOI_7_Moscow_chip152SN5_roads_test_public_AOI_7_Moscow_chip138SN5_roads_test_public_AOI_7_Moscow_chip193SN5_roads_test_public_AOI_7_Moscow_chip155SN5_roads_test_public_AOI_7_Moscow_chip96SN5_roads_test_public_AOI_7_Moscow_chip178






SN5_roads_test_public_AOI_7_Moscow_chip197SN5_roads_test_public_AOI_7_Moscow_chip132SN5_roads_test_public_AOI_7_Moscow_chip48SN5_roads_test_public_AOI_7_Moscow_chip95SN5_roads_test_public_AOI_7_Moscow_chip137SN5_roads_test_public_AOI_7_Moscow_chip115SN5_roads_test_public_AOI_7_Moscow_chip194SN5_roads_test_public_AOI_7_Moscow_chip219SN5_roads_test_public_AOI_7_Moscow_chip209
SN5_roads_test_public_AOI_7_Moscow_chip0SN5_roads_test_public_AOI_7_Moscow_chip121SN5_roads_test_public_AOI_7_Moscow_chip200

SN5_roads_test_public_AOI_7_

Process ForkPoolWorker-26:
Process ForkPoolWorker-22:
Process ForkPoolWorker-9:
Process ForkPoolWorker-17:
Process ForkPoolWorker-37:
Process ForkPoolWorker-39:
Process ForkPoolWorker-20:
Process ForkPoolWorker-21:
Process ForkPoolWorker-3:
Process ForkPoolWorker-6:
Process ForkPoolWorker-13:
Process ForkPoolWorker-12:
Process ForkPoolWorker-11:
Process ForkPoolWorker-8:
Process ForkPoolWorker-10:
Process ForkPoolWorker-4:
Process ForkPoolWorker-29:
Process ForkPoolWorker-36:
Process ForkPoolWorker-19:
Process ForkPoolWorker-2:
Process ForkPoolWorker-14:
Process ForkPoolWorker-32:
Process ForkPoolWorker-33:
Process ForkPoolWorker-28:
Process ForkPoolWorker-5:
Process ForkPoolWorker-23:
Process ForkPoolWorker-18:
Process ForkPoolWorker-16:
Process ForkPoolWorker-15:
Process ForkPoolWorker-7:
Process ForkPoolWorker-24:
Process ForkPoolWorker-38:
Process ForkPoolWorker-27:
Process ForkPoolWorker-30:
Process ForkPoolWorker-40:
Process ForkPoolWorker-35:
Process ForkPoolWorker-34:
Process F

  File "/users/PCON0003/osu10670/.conda/envs/pytorch-test/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/users/PCON0003/osu10670/.conda/envs/pytorch-test/lib/python3.6/multiprocessing/pool.py", line 108, in worker
    task = get()
  File "/users/PCON0003/osu10670/.conda/envs/pytorch-test/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/users/PCON0003/osu10670/.conda/envs/pytorch-test/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/users/PCON0003/osu10670/.conda/envs/pytorch-test/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
Traceback (most recent call last):
  File "/users/PCON0003/osu10670/.conda/envs/pytorch-test/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/users/PCON0003/osu10670/.conda/envs/pytorch-test/lib/python3.6/multiprocessing/pool.py", line 108,

  File "/users/PCON0003/osu10670/.conda/envs/pytorch-test/lib/python3.6/multiprocessing/queues.py", line 334, in get
    with self._rlock:
  File "/users/PCON0003/osu10670/.conda/envs/pytorch-test/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/users/PCON0003/osu10670/.conda/envs/pytorch-test/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/users/PCON0003/osu10670/.conda/envs/pytorch-test/lib/python3.6/multiprocessing/queues.py", line 334, in get
    with self._rlock:
  File "/users/PCON0003/osu10670/.conda/envs/pytorch-test/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/users/PCON0003/osu10670/.conda/envs/pytorch-test/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/users/PCON0003/osu10670/.conda/envs/pytorch-test/lib/python3.

  File "/users/PCON0003/osu10670/.conda/envs/pytorch-test/lib/python3.6/multiprocessing/queues.py", line 334, in get
    with self._rlock:
  File "/users/PCON0003/osu10670/.conda/envs/pytorch-test/lib/python3.6/multiprocessing/connection.py", line 379, in _recv
    chunk = read(handle, remaining)
  File "/users/PCON0003/osu10670/.conda/envs/pytorch-test/lib/python3.6/multiprocessing/pool.py", line 108, in worker
    task = get()
  File "/users/PCON0003/osu10670/.conda/envs/pytorch-test/lib/python3.6/multiprocessing/synchronize.py", line 95, in __enter__
    return self._semlock.__enter__()
  File "/users/PCON0003/osu10670/.conda/envs/pytorch-test/lib/python3.6/multiprocessing/queues.py", line 334, in get
    with self._rlock:
  File "/users/PCON0003/osu10670/.conda/envs/pytorch-test/lib/python3.6/multiprocessing/queues.py", line 334, in get
    with self._rlock:
  File "/users/PCON0003/osu10670/.conda/envs/pytorch-test/lib/python3.6/multiprocessing/queues.py", line 334, in get
    with

In [6]:
res_rows = []
# for i in range(len(results)):
#     res_rows.extend(results[i])

for i in range(len(all_files)):
    res_rows.append(process_file(all_files[i]))



SN5_roads_test_public_AOI_7_Moscow_chip101
SN5_roads_test_public_AOI_7_Moscow_chip177
SN5_roads_test_public_AOI_7_Moscow_chip134
SN5_roads_test_public_AOI_7_Moscow_chip171
SN5_roads_test_public_AOI_7_Moscow_chip87
SN5_roads_test_public_AOI_7_Moscow_chip127
SN5_roads_test_public_AOI_7_Moscow_chip120
SN5_roads_test_public_AOI_7_Moscow_chip11
SN5_roads_test_public_AOI_7_Moscow_chip98
SN5_roads_test_public_AOI_7_Moscow_chip65
SN5_roads_test_public_AOI_7_Moscow_chip14
SN5_roads_test_public_AOI_7_Moscow_chip168
SN5_roads_test_public_AOI_7_Moscow_chip164
SN5_roads_test_public_AOI_7_Moscow_chip15
SN5_roads_test_public_AOI_7_Moscow_chip56
SN5_roads_test_public_AOI_7_Moscow_chip138
SN5_roads_test_public_AOI_7_Moscow_chip136
SN5_roads_test_public_AOI_7_Moscow_chip57
SN5_roads_test_public_AOI_7_Moscow_chip178
SN5_roads_test_public_AOI_7_Moscow_chip210
SN5_roads_test_public_AOI_7_Moscow_chip83
SN5_roads_test_public_AOI_7_Moscow_chip152
SN5_roads_test_public_AOI_7_Moscow_chip223
SN5_roads_test_publi

SN5_roads_test_public_AOI_7_Moscow_chip218
SN5_roads_test_public_AOI_7_Moscow_chip196
SN5_roads_test_public_AOI_7_Moscow_chip41
SN5_roads_test_public_AOI_7_Moscow_chip123
SN5_roads_test_public_AOI_7_Moscow_chip118
SN5_roads_test_public_AOI_7_Moscow_chip43
SN5_roads_test_public_AOI_7_Moscow_chip3
SN5_roads_test_public_AOI_7_Moscow_chip165
SN5_roads_test_public_AOI_7_Moscow_chip94
SN5_roads_test_public_AOI_7_Moscow_chip36
SN5_roads_test_public_AOI_7_Moscow_chip58
SN5_roads_test_public_AOI_7_Moscow_chip45
SN5_roads_test_public_AOI_7_Moscow_chip60
SN5_roads_test_public_AOI_7_Moscow_chip84
SN5_roads_test_public_AOI_7_Moscow_chip23
SN5_roads_test_public_AOI_7_Moscow_chip21
SN5_roads_test_public_AOI_7_Moscow_chip37
SN5_roads_test_public_AOI_7_Moscow_chip170
SN5_roads_test_public_AOI_7_Moscow_chip20
SN5_roads_test_public_AOI_7_Moscow_chip142
SN5_roads_test_public_AOI_7_Moscow_chip139
SN5_roads_test_public_AOI_7_Moscow_chip111
SN5_roads_test_public_AOI_7_Moscow_chip156
SN5_roads_test_public_AOI

ValueError: 4 columns passed, passed data had 71 columns

In [10]:
row_test = []
for i in res_rows:
    row_test.extend(i)

In [12]:
res_rows = row_test

In [14]:
out_file

'./wdata/solution.csv'

In [16]:
sub = pd.DataFrame(res_rows, columns=['ImageId', 'WKT_Pix', 'length_m', 'travel_time_s'])
sub.to_csv(path.join('./wdata', out_file), index=False, header=False)

elapsed = timeit.default_timer() - t0
print('Submission file created! Time: {:.3f} min'.format(elapsed / 60))

Submission file created! Time: 0.081 min


In [17]:
sub


,ImageId,WKT_Pix,length_m,travel_time_s
0,SN5_roads_test_public_AOI_7_Moscow_chip101,"LINESTRING (32 130, 33 136, 42 176, 66 254, 70...",343.038923,27.517
1,SN5_roads_test_public_AOI_7_Moscow_chip101,"LINESTRING (293 1109, 318 1212, 332 1259, 334 ...",54.622219,5.405
2,SN5_roads_test_public_AOI_7_Moscow_chip177,"LINESTRING (589 0, 574 7, 515 105, 488 143, 46...",66.813805,9.184
3,SN5_roads_test_public_AOI_7_Moscow_chip177,"LINESTRING (461 177, 413 176, 373 178, 343 183...",32.065023,4.174
4,SN5_roads_test_public_AOI_7_Moscow_chip177,"LINESTRING (461 177, 468 182, 591 217, 633 227...",72.694431,9.695
...,...,...,...,...
9487,SN5_roads_test_public_AOI_8_Mumbai_chip177,"LINESTRING (645 661, 770 670, 776 670)",38.858847,2.855
9488,SN5_roads_test_public_AOI_8_Mumbai_chip177,"LINESTRING (0 873, 0 908, 0 1011, 0 1055)",56.633023,4.452
9489,SN5_roads_test_public_AOI_8_Mumbai_chip177,"LINESTRING (0 1055, 0 1131)",23.648954,1.906
9490,SN5_roads_test_public_AOI_8_Mumbai_chip177,"LINESTRING (949 1192, 926 1267, 925 1290, 924 ...",34.288491,1.724


In [21]:
pred_folder = './wdata/test_pred'
msk0 = cv2.imread(path.join(pred_folder, img_id + '_speed0.png'), cv2.IMREAD_UNCHANGED)
msk1 = cv2.imread(path.join(pred_folder, img_id + '_speed1.png'), cv2.IMREAD_UNCHANGED)
msk2 = cv2.imread(path.join(pred_folder, img_id + '_speed2.png'), cv2.IMREAD_UNCHANGED)
msk = np.concatenate((msk0, msk1, msk2), axis=2)

ValueError: zero-dimensional arrays cannot be concatenated

In [22]:
msk0

In [23]:
path.join(pred_folder, img_id + '_speed0.png')

'./wdata/test_pred/SN5_roads_test_public_AOI_7_Moscow_PS-MS_chip193_speed0.png'

In [ ]:
SN5_roads_test_public_AOI_7_Moscow_chip193_speed0.png